In [ ]:
!pip install --quiet syft

     |████████████████████████████████| 606 kB 5.4 MB/s 
     |████████████████████████████████| 288 kB 46.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 5.8 MB 41.5 MB/s 
     |████████████████████████████████| 6.2 MB 37.3 MB/s 
     |████████████████████████████████| 62 kB 794 kB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 25.5 MB 1.6 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 961 kB 44.0 MB/s 
     |████████████████████████████████| 15.7 MB 36.9 MB/s 
     |████████████████████████████████| 61 kB 399 kB/s 
     |████████████████████████████████| 789 kB 45.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 1.6 MB 42.0 MB/s 
     |████████████████████████████████| 10.1 MB 39.2 MB/s 
     |███████████████████

In [ ]:
# データのロード
# Load the SVHN data set
# SVHNからデータを読み込む
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import Subset

# 画像変換
# 1.Totenser関数でテンソル化
# 2.Normalise関数（平均、標準偏差）で前処理、torchvisionsの場合
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# 訓練SVHN（数字）データセット（デイレクトリ、訓練用画像とラベル、画像をテンソル化し返す、なし、ダウンロード）
train_data = datasets.SVHN('datasets/SVHN/train/', split='train', transform=transform,
                                 target_transform=None, download=True)
# テストSVHN
test_data = datasets.SVHN('datasets/SVHN/test/', split='test', transform=transform,
                               target_transform=None, download=True)

# 教師の数
num_teachers = 20  # -> 20に増やしてみる

# 悪い教師の数
num_malicious = 16 ## いろいろ試す   # 4, 8, 12 くらいで試してみる

# 1バッチに含まれるデータ数
batch_size = 64

# get_data_loders関数（訓練用データセット、教師の数）
def get_data_loaders(train_data, num_teachers):
    # 教師分類器用のデータローダー作成関数 
    """ Function to create data loaders for the Teacher classifier """
    # teacher_loders空リストの作成
    teacher_loaders = []
    # 教師あたりの訓練用データの総数知りたい
    data_size = len(train_data) // num_teachers
    print("教師の数"+str(num_teachers))
    print("悪い教師の数"+str(num_malicious))
    print("1教師あたりのデータサイズ" + str(data_size))
    print("バッチサイズ"+str(batch_size))
    print("サブセットの数"+str(data_size / batch_size))
    print("サブセットの数:1教師あたりのデータサイズ÷バッチサイズ:" + str(data_size / batch_size))
    # データ総数分くりかえす
    for i in range(data_size):
        # teacher ごとにデータを割り振る
        indices = list(range(i*data_size, (i+1)*data_size))
        subset_data = Subset(train_data, indices)
        # ここでミニバッチのためのシャッフル
        loader = torch.utils.data.DataLoader(subset_data, batch_size=batch_size, shuffle=True)
        teacher_loaders.append(loader)
    # 教師用分類器を戻す    
    return teacher_loaders
# teacher_loders教師用データ分類器の宣言
teacher_loaders = get_data_loaders(train_data, num_teachers)

Using downloaded and verified file: datasets/SVHN/train/train_32x32.mat
Using downloaded and verified file: datasets/SVHN/test/test_32x32.mat
教師の数20
悪い教師の数16
1教師あたりのデータサイズ3662
バッチサイズ64
サブセットの数57.21875
サブセットの数:1教師あたりのデータサイズ÷バッチサイズ:57.21875


In [ ]:
# 各TeacherのCNNを構築
# Define the teacher models and train them by defining a cnn
import torch.nn as nn
import torch.nn.functional as F
# 最適化
import torch.optim as optim

# CNNsの構築
# ネットワーク（classifier）をクラスとして作成、nn.Modulesを継承
class Classifier(nn.Module):
    # コンストラクタ
    def __init__(self):
        # nn.module内の_init_()を起動
        super().__init__()
        #self.は不変、convolutionやfully connectなどの学習に必要なパラメータを保持したいもの
        # 1D畳み込み
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        # convolution (畳み込み)の定義（入力チャンネル数？、畳み込み後チャンネル数、正方形フィルタ） 
        # 2D畳み込み
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # 2D特徴マップをランダムに0にする
        self.conv2_drop = nn.Dropout2d()
        #全結合層を定義する
        #fc1の第一引数は、チャネル数*最後のプーリング層の出力のマップのサイズ=特徴量の数
        # fully connectの定義（入力のサイズ（ベクトル）、出力後のベクトル）
        self.fc1 = nn.Linear(5*10*10, 50)
        self.fc2 = nn.Linear(50, 10)
    # 実際の処理、引数xがネットワークに対しての入力data、dataloaderから取得
    def forward(self, x):
        # 入力→畳み込み層1→活性化関数(ReLU)→プーリング層1(2*2)→出力
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #入力→畳み込み層2→活性化関数(ReLU)→プーリング層2(2*2)→出力
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #　
        x = x.view(x.size(0), 5*10*10)
        #入力→全結合層2→活性化関数(ReLU)→出力
        x = F.relu(self.fc1(x))
        # モデルが訓練中か推論中か、training属性
        x = F.dropout(x, training=self.training)
        #入力→全結合層2→出力
        x = self.fc2(x)
        # xが小さすぎて0を返すのを防ぐために対数とる
        return F.log_softmax(x, dim=-1)

class Classifier_small(nn.Module):
    # コンストラクタ
    def __init__(self):
        # nn.module内の_init_()を起動
        super().__init__()
        #self.は不変、convolutionやfully connectなどの学習に必要なパラメータを保持したいもの
        # 1D畳み込み
        self.conv1 = nn.Conv2d(3, 1, kernel_size=5)
        # convolution (畳み込み)の定義（入力チャンネル数？、畳み込み後チャンネル数、正方形フィルタ） 
        # 2D畳み込み
        self.conv2 = nn.Conv2d(1, 20, kernel_size=5)
        # 2D特徴マップをランダムに0にする
        self.conv2_drop = nn.Dropout2d()
        #全結合層を定義する
        #fc1の第一引数は、チャネル数*最後のプーリング層の出力のマップのサイズ=特徴量の数
        # fully connectの定義（入力のサイズ（ベクトル）、出力後のベクトル）
        self.fc1 = nn.Linear(5*10*10, 5)
        self.fc2 = nn.Linear(5, 10)
    # 実際の処理、引数xがネットワークに対しての入力data、dataloaderから取得
    def forward(self, x):
        # 入力→畳み込み層1→活性化関数(ReLU)→プーリング層1(2*2)→出力
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #入力→畳み込み層2→活性化関数(ReLU)→プーリング層2(2*2)→出力
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #　
        x = x.view(x.size(0), 5*10*10)
        #入力→全結合層2→活性化関数(ReLU)→出力
        x = F.relu(self.fc1(x))
        # モデルが訓練中か推論中か、training属性
        x = F.dropout(x, training=self.training)
        #入力→全結合層2→出力
        x = self.fc2(x)
        # xが小さすぎて0を返すのを防ぐために対数とる
        return F.log_softmax(x, dim=-1)

In [ ]:
# 各Teacherが学習モデルを作る
from tqdm import tqdm

# Defining the train and predict functions
# 学習と予測関数
# 学習の関数
def train(model, trainloader, criterion, optimizer, epochs=10, is_malicious=False):
    # 損失値の宣言
    running_loss = 0
    # エポック数だけ学習する 
    for e in tqdm(range(epochs)):
        model.train()
        # trainloaderからイテレートされたdataはimage, labelsという（配列？）なので分解      
        for images, labels in trainloader:
            images = images.to("cuda")
            labels = labels.to("cuda")
            if is_malicious:
                images *= 0.1
                # 悪い教師のラベルはランダムにする
                labels = torch.randint(0, 10, (len(labels), )).to("cuda")
            # 勾配の初期化
            optimizer.zero_grad()
            # 
            output = model.forward(images)
            # 損失関数の計算
            loss = criterion(output, labels)
            # 勾配の計算（誤差伝搬）
            loss.backward()
            # 重みの更新（最適化）
            optimizer.step()
            # 損失値
            running_loss += loss.item()


# モデルの訓練
def train_models(num_teachers):
    # モデルのリストを作る
    models = []
    # 教師の数だけループ
    for i in range(num_teachers):
        print(f"{i}-th client training...")
        # classifier をつくる
        model = Classifier().to("cuda")
        # 交差エントロピー（NNの出力をマイナス、重みの増加）
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[i], criterion, optimizer, is_malicious = (i < num_malicious))
        models.append(model)
    return models

models = train_models(num_teachers)

0-th client training...


100%|██████████| 10/10 [00:13<00:00,  1.35s/it]


1-th client training...


100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


2-th client training...


...

100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


18-th client training...


100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


19-th client training...


100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


In [ ]:
# Student用にデータをロードする
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size)

In [ ]:
# モデルの性能チェック
images, labels = next(iter(teacher_loaders[0]))
# images ... (num_samples x 3 x image_size x image_size)
# models[0](images) ... (num_samples x 10)   10 .. クラス数（最終層がlog_softmaxなので）
# models[0] ... 0番目のteacher model
output = models[0](images.cuda())   #(num_samples x 10)
# torch.argmax(value, dim=-1) ... -1の軸に対してargmaxをする。-1 ... 最後
# ようは、10この数のなかで一番大きな数のindexを返す、をnum_samplesこやる
print(torch.argmax(output, dim=-1))   # (num_samples, )
print(labels)

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
tensor([8, 5, 2, 1, 5, 3, 2, 6, 3, 1, 8, 9, 2, 1, 1, 7, 7, 3, 1, 1, 0, 4, 1, 2,
        0, 5, 2, 2, 9, 1, 5, 1, 8, 7, 4, 0, 2, 3, 1, 5, 6, 1, 1, 8, 3, 5, 5, 1,
        1, 6, 9, 1, 9, 2, 6, 4, 0, 8, 2, 4, 3, 3, 5, 2])


In [ ]:
# Next by combining the predictions of the Teacher models 
# we will generate the Aggregated Teacher and Student labels
# 次に、教師モデルの予測を組み合わせることにより、
# 集約された教師と生徒のラベルを生成します

import numpy as np

# predict関数（モデル、分類器）
# 予測結果と正解データを比較すること
def predict(model, dataloader):
    # テンソルを返す、Dタイプのデータ型返す
    outputs = torch.zeros(0, dtype=torch.long).to("cuda")
    # バッチ間の平均や分散を計算
    model.eval()

    # 分類器の中の画像とラベルのループ、詳細まだ
    for images, labels in dataloader:  # batch_size 数のimage, labelをとってくる
        images = images.to("cuda")   # (batch_size x 3 x image_size x image_size)
        labels = labels.to("cuda")   
        output = model.forward(images)  # (batch_size x 10)   ところでこのoutputはlog_softmax つまりsoftmaxの対数
        ps = torch.argmax(torch.exp(output), dim=1)   # exp(log(softmax)) = softmax  1こめの軸（=10次元の軸）についてargmaxする  ... (batch_size, ) の出力
        outputs = torch.cat((outputs, ps))


    # もともと output = [] （なにもはいってない）
    # 最初のforの中で、ps = [4,3,4] とかだったとする
    # outputs = torch.cat(([], [4,3,4])) = [4,3,4]
    # 次のループで [ 5,2,3]とかだったとする
    # outputs = torch.cat(([4,3,4], [5,2,3])) = [4,3,4,5,2,3]
    # ... というのを繰り返してくと、dataloaderに入っている画像全部についての、model.forwardの出力のargmaxがoutputに格納される
        
    return outputs

epsilon = 0.5
# 教師の統合関数（モデル、分類器、ε?何）
def aggregated_teacher(models, dataloader, epsilon):
    # テンソルを返す？
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    # 要素のインデックスと要素
    for i, model in enumerate(models):
        results = predict(model, dataloader) # ... modelsから取り出された一つのmodelでの予測結果をresultsに格納している
        preds[i] = results
    
    # 
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)

        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
    
    return preds.numpy(), labels


teacher_models = models
# 関数の実行
# 悪い教師も混ざっている場合
preds, student_labels = aggregated_teacher(teacher_models, student_train_loader, epsilon)

# いいteacherだけ使った場合（本来、誰が良いか、誰が悪いかわからないので、これはできない。誰が悪いかを見つける必要がある）

# teacher_models[num_malicious:]  -> teacher_models[4] ... teacher_models[9]  で、いいteacherが合計9-4+1=6人
# teacher_models[num_malicious:num_malicious * 2] =  teacher_models[4:4 * 2] =  teacher_models[4:8] = teacher[4] .. teacher[7] でいいteachcerが4人
# good_teacher_models = teacher_models[num_malicious:] 
# print(len(good_teacher_models))
# + teacher_models[num_malicious:num_malicious * 2]#ここのコードが分からなくなった 
# preds2, student_labels2 = aggregated_teacher(tm2, student_train_loader, epsilon)


In [ ]:
teacher_indices = np.arange(len(teacher_models))  # 全teacherのインデックス
good_teacher_indices = teacher_indices[num_malicious:]  # 良いteacherのindex
# np.random.choice(list, n) ... listの要素から重複を許してランダムにn要素を抽出

good_teacher_models = []
for i in range(len(teacher_models)):
    idx = np.random.choice(good_teacher_indices, 1).item()
    good_teacher_models.append(teacher_models[idx])
preds2, student_labels2 = aggregated_teacher(good_teacher_models, student_train_loader, epsilon)

In [1]:
# scipy ... 科学計算用のライブラリ / 特に、scipy.statsは統計計算（statistics）用のライブラリ
import scipy.stats

# scipy.stats.mode ... 列ごとに最頻値（mode）とその頻度を出力する関数
mode = scipy.stats.mode(preds)
mode_values = mode[0]  # (9000, 1)という形になっていて、実はちょっと使いにくい
mode_values = mode_values.flatten()   # flatten ... (N, 1) -> (N, )という形に変換する
mode_counts = mode[1] 
num_mismatches = np.zeros(len(teacher_models))  # teacher modelの数だけ0をの並べたベクトル

for i in range(len(mode_values)):
    mismatch = (preds[:, i] != mode_values[i]).astype(int)  # 最頻値と合致しない予測がある場合に1を返す
    num_mismatches += mismatch  # それを加算する

NameError: ignored

In [ ]:
# 最頻値から外れた確率を示している（教師分の配列）
num_mismatches / len(mode_values)

array([0.87733333, 0.91366667, 0.87755556, 0.81855556, 0.84077778,
       0.86655556, 0.88644444, 0.96077778, 0.91222222, 0.94633333,
       0.85633333, 0.85444444, 0.866     , 0.90444444, 0.89844444,
       0.90366667, 0.18877778, 0.18766667, 0.21522222, 0.18366667])

In [ ]:
print(np.bincount(preds[:, 2], minlength=10))  # 悪いの混ざってる
print(np.bincount(preds2[:, 2], minlength=10))  # 混ざってない

# この辺をちょっとデータ解析してみましょう。
# 多数決で得られた結果と違う結果を出してしまった回数、をteacherごとに出してみるとどうか？ →（多数決といつも違うことをいっている = 間違っている可能性が高い）

[3 6 1 2 1 1 3 1 1 1]
[ 0 20  0  0  0  0  0  0  0  0]


In [ ]:
print(np.bincount(preds[:, 8], minlength=10))  # 悪いの混ざってる
print(np.bincount(preds2[:, 8], minlength=10))  # 混ざってない

[3 6 2 2 1 1 2 1 1 1]
[ 0 20  0  0  0  0  0  0  0  0]


In [ ]:
# 5人より大勢が同じ答えを出す割合
print((np.array([np.bincount(p, minlength=10).max() for p in preds.T]) > 5).sum() / len(preds.T))#5から10に変更

# 正しいラベルを与える人を10人に水増ししたうえで、5人より大勢が同じ答えを出す割合
print((np.array([np.bincount(p, minlength=10).max() for p in preds2.T]) > 5).sum() / len(preds2.T))

0.3065555555555556
1.0


In [ ]:
# Now by using the labels generated previously we will create the Student model and train it
# studentモデルの性能を調べる
# Student_loaderから画像データ（X）を持ってきて、studentモデルの予測させ、とaggregated_teacherの正解データY（画像→ラベル）との精度を表示していく

def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data): (i+1)*len(data)])

student_model = Classifier().to("cuda")
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.003)
#エポック数
epochs = 20
steps = 0
running_loss = 0
for e in tqdm(range(epochs)):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels2)
    for images, labels in train_loader:
        images = images.to("cuda")
        labels = labels.to("cuda")
        steps += 1
        
        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    images = images.to("cuda")
                    labels = labels.to("cuda")
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Train Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1/20..  Train Loss: 0.790..  Test Loss: 2.212..  Accuracy: 0.216
Epoch: 1/20..  Train Loss: 0.771..  Test Loss: 2.116..  Accuracy: 0.299


  5%|▌         | 1/20 [00:03<01:15,  3.99s/it]

Epoch: 2/20..  Train Loss: 0.730..  Test Loss: 1.914..  Accuracy: 0.354
Epoch: 2/20..  Train Loss: 0.639..  Test Loss: 1.470..  Accuracy: 0.553
Epoch: 2/20..  Train Loss: 0.533..  Test Loss: 1.230..  Accuracy: 0.628


 10%|█         | 2/20 [00:08<01:14,  4.13s/it]

...

 90%|█████████ | 18/20 [01:15<00:08,  4.13s/it]

Epoch: 19/20..  Train Loss: 0.300..  Test Loss: 0.806..  Accuracy: 0.748
Epoch: 19/20..  Train Loss: 0.287..  Test Loss: 0.852..  Accuracy: 0.741
Epoch: 19/20..  Train Loss: 0.266..  Test Loss: 0.815..  Accuracy: 0.767


 95%|█████████▌| 19/20 [01:19<00:04,  4.15s/it]

Epoch: 20/20..  Train Loss: 0.324..  Test Loss: 0.806..  Accuracy: 0.745
Epoch: 20/20..  Train Loss: 0.278..  Test Loss: 0.800..  Accuracy: 0.767
Epoch: 20/20..  Train Loss: 0.273..  Test Loss: 0.778..  Accuracy: 0.779


100%|██████████| 20/20 [01:23<00:00,  4.19s/it]


In [ ]:
# Now we will perform PATE Analysis on the student labels generated 
# by the Aggregated Teacher
from syft.frameworks.torch.dp import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=student_labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

ModuleNotFoundError: ignored